In [27]:
from torch.autograd import Variable
import deepchem as dc
from deepchem.feat import Featurizer

from dataloader import *
import model
import model_old
from os import listdir
from os.path import isfile, join
from pprint import pprint
from tqdm import tqdm

# Source code for fc models (tf for classification, tf_regression for regression)
# https://github.com/deepchem/deepchem/blob/master/deepchem/models/tensorgraph/fcnet.py

In [28]:
def benchmark(dir_path, ckpt_file, tasks, _model, featurizer, metric_type):
    
    header = ckpt_file.split('.')[0]
    _ckpt = header+'_'+featurizer
    checkpoint = torch.load(join(dir_path, ckpt_file), map_location=torch.device('cuda'))
    args = checkpoint['args']
    _args = vars(args)
    args.batch_size = 1
    args.test_batch_size = 1
    args.act = 'gelu'
#     comet = model.Encoder(args)
    comet = model_old.Encoder(args)
    comet.load_state_dict(checkpoint['encoder'])
    comet.eval()

    def mol_to_graph(mol):

        adj = Chem.rdmolops.GetAdjacencyMatrix(mol)
        list_feature = list()
        for atom in mol.GetAtoms():
            list_feature.append(atom_feature(atom))
        return np.array(list_feature), adj

    class MyEncoder(Featurizer):
        name = ['comet_encoder']
        def __init__(self, model):
            self.device = 'cuda'

            self.model = model.to(self.device)

        def _featurize(self, mol):
            X, A = mol_to_graph(mol)
            X = Variable(torch.unsqueeze(torch.from_numpy(X), dim=0)).to(self.device).long()
            A = Variable(torch.unsqueeze(torch.from_numpy(A.astype(float)), dim=0)).to(self.device).float()
            _, _, molvec = self.model(X, A)
            return torch.squeeze(molvec).detach().cpu().numpy()

    if metric_type == 'reg':
        _metric = [dc.metrics.Metric(dc.metrics.mae_score, np.mean)]
    if metric_type == 'cls':
        _metric = [dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)]
    
    if featurizer == 'comet':
        feat = MyEncoder(comet)
    if featurizer == 'raw':
        feat = dc.feat.raw_featurizer.RawFeaturizer()
    if featurizer == 'fingerprint':
        feat = dc.feat.fingerprints.CircularFingerprint(size=256)
    if featurizer == 'default':
        feat = None
    

    task_result = dc.molnet.run_benchmark(
                                 ckpt = _ckpt,
                                 arg = _args,
                                 datasets = tasks,
                                 model = _model,
                                 split = None,
                                 metric = _metric,
                                 n_features = 256,
                                 featurizer = feat,
                                 out_path= './results',
                                 hyper_parameters = None,
                                 test = True,
                                 reload = False,
                                 seed = 123
                                 )

## task types
---
1) classification: muv, hiv, bace_c, bbbp, tox21, toxcast, sider, clintox


2) regression: lipo, qm7, qm8, delaney, sampl

classification = ['bace_c', 'bbbp', 'clintox', 'hiv', 'muv', 'pcba', 'sider', 'tox21', 'toxcast']
cls_models =  ['rf', 'tf', 'tf_robust', 'logreg', 'irv', 'graphconv', 'dag', 'xgb',
      'weave', 'kernelsvm']

regression = ['bace_r', 'chembl', 'clearance', 'delaney', 'hopv', 'kaggle', 'lipo',
                'nci', 'pdbbind', 'ppb', 'qm7', 'qm7b', 'qm8', 'qm9', 'sampl']
reg_models = ['tf_regression', 'tf_regression_ft', 'rf_regression', 'graphconvreg',
      'dtnn', 'dag_regression', 'xgb_regression', 'weave_regression', 'krr', 'ani']

In [31]:
cls_tasks = ['bace_c', 'bbbp', 'tox21', 'sider', 'clintox']
reg_tasks = ['qm7', 'qm8'] #['lipo', 'qm7', 'qm8', 'delaney', 'sampl']

## Fingerprint

In [ ]:
fingerprint_result = dc.molnet.run_benchmark(
                                 ckpt = 'fingerprint',
                                 arg = {'input':'fingerprint'},
                                 datasets = reg_tasks,
                                 model = 'tf_regression',
                                 split = None,
                                 metric =  [dc.metrics.Metric(dc.metrics.mae_score, np.mean)],
                                 n_features = 256,
                                 featurizer = dc.feat.fingerprints.CircularFingerprint(size=256),
                                 out_path= './benchmark',
                                 hyper_parameters = None,
                                 test = True,
                                 reload = False,
                                 seed = 123
                                 )

In [ ]:
fingerprint_result = dc.molnet.run_benchmark(
                                 ckpt = 'fingerprint',
                                 arg = {'input':'fingerprint'},
                                 datasets = cls_tasks,
                                 model = 'tf',
                                 split = None,
                                 metric = None,
                                 n_features = 256,
                                 featurizer = dc.feat.fingerprints.CircularFingerprint(size=256),
                                 out_path= './benchmark/',
                                 hyper_parameters = None,
                                 test = True,
                                 reload = False,
                                 seed = 123
                                 )

## model_8_256

In [ ]:
model_8_256 = ['model_8_256_001_000004000.tar','model_8_256_005_000020000.tar','model_8_256_010_000040000.tar',
               'model_8_256_015_000060000.tar','model_8_256_020_000080000.tar','model_8_256_025_000100000.tar' ]

In [ ]:
benchmark_models('model_8_256', model_8_256)

In [ ]:
model_4_256 = ['model_4_256_s_no_auxilary_001_000004000.tar', 'model_4_256_s_no_auxilary_002_000008000.tar', 
'model_4_256_s_no_auxilary_003_000012000.tar', 'model_4_256_s_no_auxilary_004_000016000.tar']

In [ ]:
benchmark_models('model_4_256', model_4_256)

In [ ]:
model_4_256_fc = [
'model_4_256_s_fc_001_000004000.tar',
'model_4_256_s_fc_002_000008000.tar',
'model_4_256_s_fc_003_000012000.tar',
'model_4_256_s_fc_004_000016000.tar',
'model_4_256_s_fc_005_000020000.tar',
'model_4_256_s_fc_006_000024000.tar',
'model_4_256_s_fc_007_000028000.tar',
'model_4_256_s_fc_008_000032000.tar']

In [21]:
def cls_benchmark_models(dir_path, files, tasks=['sider'], skip=3):
    print("Total {} results".format(len(files)//skip))
    for i, file in tqdm(enumerate(files)):
        if i % skip == 0:
            print("process", file)
            benchmark(dir_path, file, tasks, 'tf', 'comet', 'cls')

def reg_benchmark_models(dir_path, files, tasks=['qm8'], skip=3):
    print("Total {} results".format(len(files)//skip))
    for i, file in tqdm(enumerate(files)):
        if i % skip == 0:
            print("process", file)
            benchmark(dir_path, file, tasks, 'tf_regression', 'comet', 'reg')


In [22]:
def get_filelist(dir_path):
    file_list = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) if 'tar' in f]
    file_list.sort()
    return file_list

In [26]:
dir_path = './runs/model_4_256_s_fc_train'
model_file_list = get_filelist(dir_path)
# cls_benchmark_models(dir_path, model_file_list, tasks=['tox21'])
reg_benchmark_models(dir_path, model_file_list, tasks=['qm8'])



0it [00:00, ?it/s]

Total 7 results
process model_4_256_s_fc_001_000004000.tar
-------------------------------------
Benchmark on dataset: qm8
-------------------------------------
Loading raw samples now.
shard_size: 8192
About to start loading CSV from /tmp/qm8.csv
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample 3000
Featurizing sample 4000
Featurizing sample 5000
Featurizing sample 6000
Featurizing sample 7000
Featurizing sample 8000
TIMING: featurizing shard 0 took 99.195 s
Loading shard 2 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample 3000
Featurizing sample 4000
Featurizing sample 5000
Featurizing sample 6000
Featurizing sample 7000
Featurizing sample 8000
TIMING: featurizing shard 1 took 99.408 s
Loading shard 3 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample 3000
Featurizing sample 4000
Featurizing sample 5000
TIMING: fea



1it [05:58, 358.79s/it]

computed_metrics: [0.009564908059126001, 0.0094294156278122, 0.01626261029803162, 0.032372075818478965, 0.009726381646059922, 0.009307862499316762, 0.012846826816409213, 0.02457030190703601, 0.009750409586901682, 0.009328142506072232, 0.012816028100452166, 0.024576254862306676, 0.009086381581742927, 0.008734133222058642, 0.015649619733733425, 0.028698140324061833]
process model_4_256_s_fc_004_000016000.tar
-------------------------------------
Benchmark on dataset: qm8
-------------------------------------
Loading raw samples now.
shard_size: 8192
About to start loading CSV from /tmp/qm8.csv
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample 3000
Featurizing sample 4000
Featurizing sample 5000
Featurizing sample 6000
Featurizing sample 7000
Featurizing sample 8000
TIMING: featurizing shard 0 took 99.353 s
Loading shard 2 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sampl



4it [12:02, 287.48s/it]

...



7it [18:07, 237.78s/it]

computed_metrics: [0.009352418063662373, 0.009702554961608859, 0.01640175301003344, 0.033557261807831945, 0.009699419072447278, 0.009594723399957433, 0.015423485178659452, 0.02749364721137895, 0.009696420781674897, 0.009592068290309836, 0.01541425840540688, 0.027496639376314472, 0.00904610649018594, 0.009000533932317869, 0.016417793781930767, 0.02939575808006283]
process model_4_256_s_fc_010_000040000.tar
-------------------------------------
Benchmark on dataset: qm8
-------------------------------------
Loading raw samples now.
shard_size: 8192
About to start loading CSV from /tmp/qm8.csv
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample 3000
Featurizing sample 4000
Featurizing sample 5000
Featurizing sample 6000
Featurizing sample 7000
Featurizing sample 8000
TIMING: featurizing shard 0 took 99.150 s
Loading shard 2 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample



10it [24:15, 203.29s/it]

computed_metrics: [0.009284849408250577, 0.009731696310265863, 0.015935331271720674, 0.03207175932727116, 0.009251951546989389, 0.009598604586160504, 0.013767920620134075, 0.02498712929488009, 0.009246113459439077, 0.009604057128537695, 0.013767217178523973, 0.02500838713222565, 0.008795766371793088, 0.00917263887514384, 0.015571016709400278, 0.027887338242222226]
process model_4_256_s_fc_013_000052000.tar
-------------------------------------
Benchmark on dataset: qm8
-------------------------------------
Loading raw samples now.
shard_size: 8192
About to start loading CSV from /tmp/qm8.csv
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample 3000
Featurizing sample 4000
Featurizing sample 5000
Featurizing sample 6000
Featurizing sample 7000
Featurizing sample 8000
TIMING: featurizing shard 0 took 99.674 s
Loading shard 2 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sampl

KeyboardInterrupt: 

In [29]:
dir_path = './runs/model_8_256_train'
model_file_list = get_filelist(dir_path)
cls_benchmark_models(dir_path, model_file_list, tasks=['sider'])
# reg_benchmark_models(dir_path, model_file_list, tasks=['qm8'])




0it [00:00, ?it/s]

Total 8 results
process model_8_256_001_000004000.tar
-------------------------------------
Benchmark on dataset: sider
-------------------------------------
Loading raw samples now.
shard_size: 8192
About to start loading CSV from /tmp/sider.csv.gz
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
TIMING: featurizing shard 0 took 31.311 s
TIMING: dataset construction took 31.387 s
Loading dataset from disk.
TIMING: dataset construction took 0.082 s
Loading dataset from disk.
TIMING: dataset construction took 0.065 s
Loading dataset from disk.
TIMING: dataset construction took 0.020 s
Loading dataset from disk.
TIMING: dataset construction took 0.029 s
Loading dataset from disk.





1it [00:33, 33.45s/it]

computed_metrics: [0.6615958102279729, 0.6188332215729476, 0.7792006754855052, 0.6401828500616311, 0.6463294517224649, 0.6355700897944963, 0.7209765602279998, 0.6552251678748178, 0.6284469065974508, 0.648802932998696, 0.6465173739116794, 0.6188926818860678, 0.662612919792219, 0.6466321243523316, 0.6245822351438741, 0.6813322368421053, 0.6892389424264889, 0.6134361344115726, 0.604423557495058, 0.6356802102284212, 0.657777116750185, 0.6473157385472623, 0.6754182754182754, 0.6323379958979286, 0.6648711169861203, 0.7418959330143541, 0.6124430169912971]
computed_metrics: [0.6441851265822787, 0.49905704856199906, 0.6830985915492958, 0.5433826469412235, 0.5927873779113448, 0.5927318295739348, 0.5692137320044297, 0.5430508474576272, 0.5392287234042553, 0.6127508854781581, 0.5596846846846847, 0.5695592286501377, 0.6386554621848739, 0.5040650406504066, 0.5498525073746312, 0.6118332586284178, 0.6033464566929134, 0.5788446526151444, 0.6451162790697674, 0.5429033998920669, 0.5437209302325581, 0.568




4it [01:06, 26.74s/it]

...

computed_metrics: [0.6742914356130623, 0.6160514932906924, 0.7733370860305845, 0.6443651991641892, 0.6381052886948083, 0.6238866426150016, 0.7215000290816029, 0.6496088054705629, 0.6311858365939642, 0.6446138136364755, 0.6386344026982509, 0.6204928525709409, 0.6683167900049671, 0.6771431935939708, 0.6112681031437655, 0.686962047534845, 0.6952063256733383, 0.6465492993118691, 0.6084181865652158, 0.6158055387103296, 0.6376221312038075, 0.6480773080644623, 0.7058742415885273, 0.6301822526900596, 0.6570904017037527, 0.7174441786283892, 0.6224409448818897]
computed_metrics: [0.6281645569620253, 0.4884488448844884, 0.7112676056338028, 0.5249900039984006, 0.6149511645379413, 0.5152882205513785, 0.566998892580288, 0.5315254237288136, 0.544104609929078, 0.6107831562377017, 0.5737612612612611, 0.4511019283746556, 0.6446078431372548, 0.5317073170731708, 0.5339233038348082, 0.6544150605109815, 0.6486220472440944, 0.5569867291178766, 0.5490697674418604, 0.5045871559633028, 0.5053488372093023, 0.611




22it [04:27, 13.01s/it]

computed_metrics: [0.6817629390018485, 0.6264710000221063, 0.7655033305188105, 0.6512006538834216, 0.637493934983018, 0.6339077664617194, 0.7291484906648054, 0.6538185103805163, 0.6311393483903459, 0.6520847174036072, 0.6345399639187386, 0.6456688713462769, 0.6765589876998459, 0.6520960904380593, 0.6155929317356374, 0.6953970121697525, 0.6878675562144798, 0.6320608769903059, 0.6212226325015852, 0.6317828987265008, 0.6428925550496152, 0.658139534883721, 0.6849880492737636, 0.6303885881113842, 0.6696151868987295, 0.7300189393939394, 0.6235426163834784]
computed_metrics: [0.6635680379746836, 0.5794436586515794, 0.823943661971831, 0.5561775289884046, 0.6716754320060105, 0.5260651629072681, 0.5681063122923589, 0.512542372881356, 0.5378989361702127, 0.6499409681227863, 0.5498310810810811, 0.6260330578512396, 0.6358543417366946, 0.49959349593495933, 0.5014749262536873, 0.6671896010757508, 0.6402559055118111, 0.539032006245121, 0.5520930232558139, 0.5593631948192122, 0.5702325581395349, 0.6502




25it [05:00, 12.40s/it]




computed_metrics: [0.6879975354282193, 0.6360025643481912, 0.7665353222628764, 0.6539526330013381, 0.6337481804949054, 0.6402485500039925, 0.7378245022198957, 0.6627102285561739, 0.6438345019951187, 0.6533396077852416, 0.6506245587889246, 0.6161635018846454, 0.6782975105026983, 0.6614107395195479, 0.606350025813113, 0.7069654280216351, 0.690449715838893, 0.6336882816816515, 0.6324792062959232, 0.6369820092985647, 0.6418094867629225, 0.6621901171132094, 0.7016731016731017, 0.6354114697857192, 0.6575420430344422, 0.7406598883572568, 0.6245959386655615]
computed_metrics: [0.6615901898734178, 0.5304101838755304, 0.8309859154929577, 0.5723710515793683, 0.6040570999248686, 0.5338345864661654, 0.5509413067552602, 0.5372881355932203, 0.53302304964539, 0.6188508461235733, 0.5230855855855856, 0.559228650137741, 0.5927871148459384, 0.49796747967479676, 0.5448377581120945, 0.6613626176602421, 0.6062992125984252, 0.5362997658079625, 0.6313953488372093, 0.5860766324878575, 0.5197674418604651, 0.6131

In [39]:
dir_path = './runs/model_4_256_s_no_auxilary_train'
model_file_list = get_filelist(dir_path)
print(model_file_list)
cls_benchmark_models(dir_path, [model_file_list[-1]], tasks=['sider', 'tox21'])
reg_benchmark_models(dir_path, [model_file_list[-1]], tasks=['qm8', 'qm7'])




0it [00:00, ?it/s]

['model_4_256_s_no_auxilary_001_000004000.tar', 'model_4_256_s_no_auxilary_002_000008000.tar', 'model_4_256_s_no_auxilary_003_000012000.tar', 'model_4_256_s_no_auxilary_004_000016000.tar', 'model_4_256_s_no_auxilary_005_000020000.tar', 'model_4_256_s_no_auxilary_006_000024000.tar']
Total 0 results
process model_4_256_s_no_auxilary_006_000024000.tar
-------------------------------------
Benchmark on dataset: sider
-------------------------------------
Loading raw samples now.
shard_size: 8192
About to start loading CSV from /tmp/sider.csv.gz
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
TIMING: featurizing shard 0 took 17.236 s
TIMING: dataset construction took 17.315 s
Loading dataset from disk.
TIMING: dataset construction took 0.084 s
Loading dataset from disk.
TIMING: dataset construction took 0.068 s
Loading dataset from disk.
TIMING: dataset construction took 0.022 s
Loading dataset from disk.
TIMING: dataset construction took 0.022 s
Loading dataset f




1it [01:56, 116.30s/it]





0it [00:00, ?it/s]

computed_metrics: [0.8496779086659647, 0.8913199462106616, 0.8836952079632604, 0.8263722591606577, 0.7379901444714105, 0.8250580986290856, 0.8563595517112346, 0.7735099964014629, 0.8439690151235706, 0.8011255486355673, 0.857122644563204, 0.8184145829837222]
computed_metrics: [0.7628259041211102, 0.789021164021164, 0.8415783006416967, 0.7586426029486527, 0.6563636363636363, 0.7325938163795138, 0.7398753894080996, 0.7605157471993236, 0.7510713444784554, 0.656665767709457, 0.8063666195597499, 0.7733850129198967]
computed_metrics: [0.8035417819590481, 0.8191985612319452, 0.894658279134236, 0.7765333891563744, 0.7043072833830804, 0.7375208541875209, 0.7560410460112545, 0.7252862423368194, 0.7818083010665036, 0.725609756097561, 0.8288524692384436, 0.7973484848484849]
Total 0 results
process model_4_256_s_no_auxilary_006_000024000.tar
-------------------------------------
Benchmark on dataset: qm8
-------------------------------------
Loading raw samples now.
shard_size: 8192
About to start l




1it [07:35, 455.23s/it]




computed_metrics: [109.01008632797965]
computed_metrics: [110.73501766565828]
computed_metrics: [113.36072444954212]


In [41]:
dir_path = './runs/model_8_256_train'
model_file_list = get_filelist(dir_path)
print(model_file_list)
cls_benchmark_models(dir_path, [model_file_list[-1]], tasks=['sider', 'tox21'])
reg_benchmark_models(dir_path, [model_file_list[-1]], tasks=['qm8', 'qm7'])




0it [00:00, ?it/s]

['model_8_256_001_000004000.tar', 'model_8_256_002_000008000.tar', 'model_8_256_003_000012000.tar', 'model_8_256_004_000016000.tar', 'model_8_256_005_000020000.tar', 'model_8_256_006_000024000.tar', 'model_8_256_007_000028000.tar', 'model_8_256_008_000032000.tar', 'model_8_256_009_000036000.tar', 'model_8_256_010_000040000.tar', 'model_8_256_011_000044000.tar', 'model_8_256_012_000048000.tar', 'model_8_256_013_000052000.tar', 'model_8_256_014_000056000.tar', 'model_8_256_015_000060000.tar', 'model_8_256_016_000064000.tar', 'model_8_256_017_000068000.tar', 'model_8_256_018_000072000.tar', 'model_8_256_019_000076000.tar', 'model_8_256_020_000080000.tar', 'model_8_256_021_000084000.tar', 'model_8_256_022_000088000.tar', 'model_8_256_023_000092000.tar', 'model_8_256_024_000096000.tar', 'model_8_256_025_000100000.tar']
Total 0 results
process model_8_256_025_000100000.tar
-------------------------------------
Benchmark on dataset: sider
-------------------------------------
Loading raw samp




1it [03:30, 210.46s/it]





0it [00:00, ?it/s]

computed_metrics: [0.8415428465744517, 0.8862235067437378, 0.8575353924260067, 0.7951018933841447, 0.7281757113674732, 0.8148050231328487, 0.8265620246758038, 0.7362271430125614, 0.8236217054209614, 0.7767671703415853, 0.8244330731902234, 0.7992463780276333]
computed_metrics: [0.7635185276802058, 0.8000992063492063, 0.8183452418877712, 0.7015760040671073, 0.6780909090909091, 0.7425851624577138, 0.7446348217376255, 0.7593414594048709, 0.7868142218036261, 0.7019732110751529, 0.7628685717023163, 0.788975021533161]
computed_metrics: [0.7895591219332225, 0.8861349969088967, 0.8815438746134983, 0.7628741888214361, 0.7294167628924377, 0.7904738071404738, 0.655246607083747, 0.7008992967904797, 0.7914543848923308, 0.6984591298615689, 0.7865345443964084, 0.7820583887657059]
Total 0 results
process model_8_256_025_000100000.tar
-------------------------------------
Benchmark on dataset: qm8
-------------------------------------
Loading raw samples now.
shard_size: 8192
About to start loading CSV 




1it [12:29, 749.38s/it]




computed_metrics: [108.55753549618473]
computed_metrics: [106.97330430701801]
computed_metrics: [105.67685971891085]
